See auxiliary code at <https://github.com/bsmith89/swc-instructor-training-analysis> and pre-computed results at <https://bsmith89.github.io/swc-instructor-training-analysis/>.

In [ ]:
import pandas as pd
import numpy as np
import patsy
import statsmodels.api as sm
import matplotlib.pyplot as plt

%matplotlib inline

def get_person_details(data):
    data = data.sort_values('Taught')
    certified = data.Certified.drop_duplicates()
    assert len(certified) == 1
    taught = data.Taught.drop_duplicates()
    
    try:
        taught_first = taught.iloc[0]
    except IndexError:
        taught_first = np.nan
    try:
        taught_second = taught.iloc[1]
    except IndexError:
        taught_second = np.nan
    
    return pd.Series({'certified': certified.iloc[0],
                      'taught_first': taught_first,
                      'taught_second': taught_second,
                      'taught_count': taught.notnull().sum()})

In [ ]:
raw_data = (pd.read_csv('teaching-stats-2016-05.csv')
              .sort_values(['Person', 'Taught', 'Certified'])
              .reindex())

# Since I don't know exactly which day in May the data was collected,
# I'm using June 1st, 2016 as the date for right censoring.
COLLECTION_DATE = pd.datetime(year=2016, month=6, day=1)

data = (raw_data.groupby('Person')
                .apply(get_person_details))
data.certified = pd.to_datetime(data.certified)
data.taught_first = pd.to_datetime(data.taught_first)
data.taught_second = pd.to_datetime(data.taught_second)
data['has_taught'] = data.taught_first.notnull()
data['has_taught_multiple'] = data.taught_second.notnull()

data['time_to_taught_first'] = (data.taught_first - data.certified).dt.days
data['time_to_taught_second'] = (data.taught_second - data.certified).dt.days
data['time_since_certified'] = (COLLECTION_DATE - data.certified).dt.days
data['time_since_taught_first'] = (COLLECTION_DATE - data.taught_first).dt.days
data['time_between_first_second'] = data.time_to_taught_second - data.time_to_taught_first
data['year_certified'] = data.certified.dt.year

# Drop individuals who taught a workshop before instructor training.
data = data[(data.time_to_taught_first > 0) | data.time_to_taught_first.isnull()]

## Survival analysis of time to first teaching

In [ ]:
data.time_to_taught_first.plot.hist(bins=data.taught_count.max())
print("{} of {} instructors have not yet taught."
          .format(sum(~data.has_taught), len(data)))
plt.xlabel('Days between certification and first teaching')

In [ ]:
_data = data.copy()

# Fill in dates for right censoring
_data.time_to_taught_first.fillna(_data.time_since_certified, inplace=True)

# Fit a proportional hazards model, comparing certification year.
# "Sum" stands for sum-to-zero coding for the design matrix.
ydm, xdm = patsy.dmatrices('time_to_taught_first ~ C(year_certified, Sum)',
                           data=_data, return_type='dataframe')
xdm = xdm.drop('Intercept', axis='columns')  # Remove the intercept term

# Right censor for individuals who have not yet taught by the date
# of this data collection.
fit = sm.PHReg(ydm, xdm, status=_data.has_taught).fit()

# I believe that, given the sum-to-zero coding in the model,
# the "baseline" cumulative hazard function should represent
# the mean of annual means.
sf = fit.baseline_cumulative_hazard[0]
plt.plot(sf[0], sf[2])
plt.ylim(0, 1)

# No certification year was significantly different from the overall mean.
fit.summary()

## Time to second workshop

In [ ]:
data.time_between_first_second.plot.hist()
print('Of {} instructors who have taught, {} have taught a second time.'
          .format(sum(data.has_taught), sum(data.taught_second.notnull())))
plt.xlabel('Days between first teaching and second')

In [ ]:
_data = data.copy()
_data = _data[_data.time_to_taught_first.notnull()]

# Fill in dates for right censoring.
_data.time_between_first_second.fillna(_data.time_since_taught_first, inplace=True)

# Fit a proportional hazards model using time between certification and first taught.
ydm, xdm = patsy.dmatrices('time_between_first_second ~ time_to_taught_first',
                           data=_data, return_type='dataframe')
xdm = xdm.drop('Intercept', axis='columns')  # Remove the intercept term

# Right censor for individuals who have not yet taught a second time.
fit = sm.PHReg(ydm, xdm, status=_data.has_taught_multiple).fit()

# The probability of having not taught a second time for someone who taught
# at day 0 of being certified.
sf = fit.baseline_cumulative_hazard[0]
plt.plot(sf[0], sf[2])
plt.ylim(0, 1)

# The greater the gap between certification and teaching the first time
# the lower the rate of teaching a second time.
print(("The per-day chance of teaching again goes down "
       "by {:.2}% for every day "
       "between certification and teaching the first time.")
           .format(-fit.params[0] * 100))
fit.summary()

## Changelog

In [ ]:
!git log